In [7]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# Split set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Path to image folders, 
data_path = fr'../Datasets/train'
# os.listdir(data_path)

def remove_white_background(pixels):
    for pixel in pixels:
        if ((256 > pixel > 200)):
            pixel = 0

    return pixels

def get_rgb_pixels_onehut_labels(src):
    print("Starting...")
    newPixels = []
    y = np.empty(shape=[0, 1])

    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            grey_img = img.convert('L')
            imgResize = grey_img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            newPixels.append(pixels)
            y = np.append(y, subdir)
    return newPixels, LabelBinarizer().fit_transform(y) # OneHot encode y

def process_files(src):
    X_grey_train = []
    X_grey_validation = []
    X_grey_test = []
    all_pixels, y = get_rgb_pixels_onehut_labels(src)

    X_train, X_validation, y_train, y_validation = train_test_split(all_pixels, y, test_size=0.2, random_state=1)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
    
    for pixels in X_train:       
        X_grey_train.append(pixels.copy())
        
    for pixels in X_validation:       
        X_grey_validation.append(pixels.copy())
        
    for pixels in X_test:       
        X_grey_test.append(pixels.copy())
    
    print("Finished")
    return np.asarray(X_grey_train), np.asarray(X_grey_validation), np.asarray(X_grey_test), y_train, y_validation, y_test

In [8]:
# Call process_files and assign variables
X_grey_train, X_grey_validation, X_grey_test, y_train, y_validation, y_test = process_files(data_path)

Starting...
Finished


In [9]:
print(X_grey_train)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [254 254 254 ... 255 255 255]]


In [11]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

# Call process_files and assign variables
X_grey_train, X_grey_validation, X_grey_test, y_train, y_validation, y_test = process_files(data_path)
# Fit/train train-datasets and store prediction vectors in variables

# Grey
mlpc_grey =  MLPClassifier(hidden_layer_sizes=(576), activation='relu', solver='adam')
mlpc_grey.fit(X_grey_train, y_train)
grey_val_result = mlpc_grey.predict(X_grey_validation)

print("Grey model")
print(classification_report(y_validation, grey_val_result))

Starting...
Finished
Grey model
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        79
           1       1.00      0.99      0.99        91
           2       0.80      1.00      0.89       100
           3       1.00      0.93      0.96        69
           4       1.00      1.00      1.00        93
           5       1.00      1.00      1.00       145

   micro avg       0.96      0.98      0.97       577
   macro avg       0.97      0.98      0.97       577
weighted avg       0.97      0.98      0.97       577
 samples avg       0.96      0.98      0.97       577



c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
